<h1>Desafio Tecnico</h1>

<h2>1. Programatico</h2>

<p>Importo las bilbiotecas que use a lo largo del challenge. Como estoy usando Anaconda deberian importar el archivo rocking.yml que va a tener las variables de entorno que use.</p>

In [1]:
import boto3
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

<h3>Postman</h3>

<p>Basicamente aca muestro lo que devuelve el endpoint porque no voy a poder mostar como funciona la funcion extraer</p>

<img src='fotos/Postman.png'>

<h3>1- Programatico</h3>

<p>Funciono, ahora voy a importar varias bilbiotecas que voy a usar a lo largo del ejercicio</p>

In [2]:
# import boto3
# import os

<p>La idea es trabajar con variables de entorno para evitar hardcodear secretos.</p>

In [3]:
#pruebo las keys si estan funcionando
KEY = os.environ.get('AWS_KEY_RKD')
SECRET = os.environ.get('AWS_SECRET_RKD')

<p>Ahora voy a crear la funcion descargar</p>

In [ ]:
#Recordar que los secret estan definidos como variables de entorno
def descargar(bucket, key):
    """
    Descarga un archivo desde el servicio AWS S3 usando boto3.

    Parámetros:
    - bucket: el nombre del bucket de S3 donde se encuentra el archivo.
    - key: el identificador del archivo dentro del bucket.
    - filename: el nombre del archivo local donde se guardará el archivo descargado.

    Retorna:
    - El nombre del archivo local si se ha descargado correctamente, o None en caso contrario.
    """
    # Crear un cliente con las credenciales
    s3 = boto3.client('s3', aws_access_key_id=KEY, aws_secret_access_key=SECRET)
    # Intentar descargar el archivo desde el bucket y el key especificados
    
    # Creo el nombre del archivo local a partir del key
    filename = os.path.basename(key)

    #Verifico si el archivo ya existe y si es asi lo borro
    if os.path.exists(filename):
        os.remove(filename)
    
    try:
        with open(filename, 'wb') as f:
            s3.download_fileobj(bucket, key, f)
            # Retorno el nombre del archivo descargado
            print(f'Archivo {filename} descargado correctamente.')
            return filename
    except Exception as e:
        # Mostrar un mensaje de error con la excepción ocurrida
        print(f'Error al descargar el archivo: {e}')
        # Retornar None si ocurre una excepción
        return None

In [ ]:
#Aca genero la lista de keys que quiero descargar
bucket = 'desafio-XXX'
keys = ['disney_plus_titles.csv', 'netflix_titles.csv']

In [ ]:
#* Descargar los archivos desde el bucket desafio-rkd (tener en cuenta que si se pasa un tercer 
# parametro se sigue pudiendo modificar el nombre del archivo).

for key in keys:
    descargar(bucket, key)

<p>El output para esto en mi codigo original es esto, no quiero filtrar los secrets de la empresa por lo tanto no van a poder probar esta funcion<p>

<img src='fotos/Output.png'>

<h2>2. QA</h2>

<p>Para esta parte voy a utilizar pandas, la idea es ver las dimensiones, explorar un poco los tipos de datos de las columnas, etc para identificar los problemas respecto a calidad de los datos</p>
<p>Lo primero seria leer los csv que descargue previamente</p>

In [ ]:
# import pandas as pd

<h3>Limpieza</h3>

<h4>Disney+</h4>

In [ ]:
disney = pd.read_csv("disney_plus_titles.csv", sep=",", on_bad_lines='warn', verbose=True)

In [ ]:
disney.count()

In [ ]:
print(disney.info())

In [ ]:
print(disney.describe())

In [ ]:
#Empiezo a trabajar los datos de Disney
disney.isnull().sum()

<p>Tengo valores vacios en director, cast, country, fecha agregada y rating</p>

In [ ]:
disney[disney.duplicated()]

<p>No tengo peliculas duplicadas</p>

<p>Al analizar los datos recibia incoherencias por lo tanto decidi eliminar todas las peliculas que contengan valores nulos</p>

In [ ]:
contadordisney = disney.shape[0]
disney = disney.dropna(thresh=10,axis=0)

disney.shape[0]
contadordisneynuevo = contadordisney - disney.shape[0]
print(f'Limpie {contadordisneynuevo} peliculas con 2 valores nulos y quedaron {contadordisney} peliculas')

<p>A modo de ejemplo, puedo eliminar las peliculas que no tienen rating porque no serian relevantes para el analisis</p>

In [ ]:
disney = disney.dropna(subset=['rating'])
disney.isnull().sum()

<h4>Netflix</h4>

<p>Luego de analizar los datos puedo asegurar que es exactamente lo mismo que Disney, con la diferencia de que esta separado por ; en vez de ,</p>
<p>Por lo tanto no vale la pena describir lo que hice aca ya que fue el mismo procedimiento</p>

In [ ]:
netflix = pd.read_csv("netflix_titles.csv", sep=";", on_bad_lines='warn', verbose=True)

In [ ]:
netflix.head()

In [ ]:
netflix.count()

In [ ]:
print(netflix.info())

In [ ]:
print(netflix.describe())

In [ ]:
netflix.isnull().sum()

In [ ]:
netflix[netflix.duplicated()]

In [ ]:
contadornetflix = netflix.shape[0]
netflix = netflix.dropna(thresh=11,axis=0)

netflix.shape[0]
contadornetflixnuevo = contadornetflix - netflix.shape[0]
print(f'Limpie {contadornetflixnuevo} peliculas con 2 valores nulos y quedaron {contadornetflix} peliculas')

<h3>Analisis</h3>

In [ ]:
# import matplotlib.pyplot as plt

<h4>Analizo Disney+</h4>

<p>Voy a imprimir que tipos de contenido hay en Disney+:</p>

In [ ]:
dtype = disney["type"].value_counts()
dtype = dtype.sort_values(ascending=False)

print(dtype)                                                                                                                                                                                            

<p>Con estos datos puedo crear un grafico de barras sencillo</p>

In [ ]:
plt.bar(dtype.index, dtype.values)
#Nombre
plt.title("Frecuencia de tipos de contenido en Disney+")
#Eje X = Tipo
plt.xlabel("Tipo")
#Eje Y = Frecuencia
plt.ylabel("Frecuencia")

plt.show()

In [ ]:
ddirector = disney["director"].value_counts()
ddirector = ddirector.sort_values(ascending=False)

print(ddirector)

In [ ]:
print(f"El director con más trabajos es {ddirector.index[0]}, con {ddirector.values[0]} registros.")

<p>Realmente tiene poco sentido realizar este tipo de analisis para todos los datos, seria muy repetitivo, ahora voy a comparar Netflix con Disney+ y voy a imprimir algunos resultados como para sacar alguna conclusion mas adelante</p>

<h4>Analizo ambas tablas</h4>

<p>Se me ocurrio un analizar ambas tablas, para eso voy a crear una columna extra en cada DataFrame y concatenarlas, y asi poder comparar datos de ambas plataformas</p>

In [ ]:
disney['platform'] = 'Disney+'
netflix['platform'] = 'Netflix'

#Combino (concateno) ambas tablas, tengo que ignorar los indices
plataformas = pd.concat([disney, netflix], ignore_index=True)

<p>Hacer el mismo grafico que hice arriba en matplotlib es muy complicado por la forma en la que funciona esa libreria, voy a usar otra libreria que hace lo mismo pero mas sencillo</p>

In [ ]:
# import seaborn as sns

<p>Llegado este punto tuve que revisar como habia limpiado los datos porque encontre incoherencias</p>

In [ ]:
# Primero genero el indice agrupando los datos por type y platform
type_plataformas = plataformas.groupby(["type", "platform"]).size()
# Convertir el índice de nivel platform en columnas
type_plataformas = type_plataformas.unstack(level="platform")

# Genera un grafico usando las columnas como etiquetas del eje x y los valores como alturas de las barras
type_plataformas.plot.bar()
plt.show()

In [ ]:
#Primero me aseguro de no tener duplicados
directores = plataformas.drop_duplicates(subset=["title", "type"])
#Agrupo por director y cuento la cantidad de shows
directores = directores.groupby("director")["show_id"].count()
# Ordeno de forma ascendente y muestro los primeros 5
top_5 = directores.sort_values(ascending=False).head(5)

print(top_5)

<h2>3. Modelado de datos</h2>

<p>Hay que tener en cuenta que para el analisis de los datos agregue una columna a cada DataFrame para poder identificar si pertenece a Netflix o Disney+, es ideal esta columna para el modelado de datos.</p>

<p>Para modelar esta tabla con un esquema copo de nieve seria suficiente ya que voy a usar una tabla central que seria show y siete tablas: platform, director, cast, country, rating, listed_in y type, con las claves primarias y foraneas correspondientes, a demas agregue algunos indices para optimizar el rendimiento de las consultas *ver DER adjunto*</p>

In [ ]:
# import sqlite3

<h3>Creo la base de datos streaming.db</h3>

In [ ]:
base_datos = 'streaming.db'

if os.path.exists(base_datos):
    os.remove(base_datos)

conn = sqlite3.connect(base_datos)
cursor = conn.cursor()

<p>Si la tabla existe la elimino para el caso de ejecutar varias veces el codigo, luego crea las tablas dentro de la base de datos</p>

<h3>Creacion / comprobacion tablas SQL</h3>

In [ ]:
eliminar_tablas = [
    "DROP TABLE IF EXISTS show;",
    "DROP TABLE IF EXISTS platform;",
    "DROP TABLE IF EXISTS showXplatform;",
    "DROP TABLE IF EXISTS director;",
    "DROP TABLE IF EXISTS showXdirector;",
    "DROP TABLE IF EXISTS cast;",
    "DROP TABLE IF EXISTS showXcast;",
    "DROP TABLE IF EXISTS country;",
    "DROP TABLE IF EXISTS showXcountry;",
    "DROP TABLE IF EXISTS rating;",
    "DROP TABLE IF EXISTS showXrating;",
    "DROP TABLE IF EXISTS listed_in;",
    "DROP TABLE IF EXISTS showXlisted_in;",
    "DROP TABLE IF EXISTS type;",
    "DROP TABLE IF EXISTS showXtype;"
]
    
for tabla_sql in eliminar_tablas:
    cursor.execute(tabla_sql)

crear_tablas = [
    """
    CREATE TABLE show (
      show_id INTEGER PRIMARY KEY,
      title TEXT,
      date_added TEXT,
      duration TEXT,
      description TEXT
    );
    """,
    """
    CREATE TABLE platform (
      platform_id INTEGER PRIMARY KEY,
      platformname TEXT
    );
    """,
    """
    CREATE TABLE showXplatform (
      show_id INTEGER REFERENCES show(show_id),
      platform_id INTEGER REFERENCES platform(platform_id),
      PRIMARY KEY (show_id, platform_id)
    );
    """,
    """
    CREATE TABLE director (
      director_id INTEGER PRIMARY KEY,
      tablename TEXT
    );
    """,
    """
    CREATE TABLE showXdirector (
      show_id INTEGER REFERENCES show(show_id),
      director_id INTEGER REFERENCES director(director_id),
      PRIMARY KEY (show_id, director_id)
    );
    """,
    """
    CREATE TABLE cast (
      cast_id INTEGER PRIMARY KEY,
      castname TEXT
    );
    """,
    """
    CREATE TABLE showXcast (
      show_id INTEGER REFERENCES show(show_id),
      cast_id INTEGER REFERENCES cast(cast_id),
      PRIMARY KEY (show_id, cast_id)
    );
    """,
    """
    CREATE TABLE country (
      country_id INTEGER PRIMARY KEY,
      countryname TEXT
    );
    """,
    """
    CREATE TABLE showXcountry (
      show_id INTEGER REFERENCES show(show_id),
      country_id INTEGER REFERENCES country(country_id),
      PRIMARY KEY (show_id, country_id)
    );
    """,
    """
    CREATE TABLE rating (
      rating_id INTEGER PRIMARY KEY,
      ratingname TEXT
    );
    """,
    """
    CREATE TABLE showXrating (
      show_id INTEGER REFERENCES show(show_id),
      rating_id INTEGER REFERENCES ratingr(rating_id),
      PRIMARY KEY (show_id, rating_id)
    );
    """,
    """
    CREATE TABLE listed_in (
      listed_in_id INTEGER PRIMARY KEY,
      listed_inname TEXT
    );
    """,
    """
    CREATE TABLE showXlisted_in (
      show_id INTEGER REFERENCES show(show_id),
      listed_in_id INTEGER REFERENCES listed_in(listed_in_id),
      PRIMARY KEY (show_id, listed_in_id)
    );
    """,
    """
    CREATE TABLE type (
      type_id INTEGER PRIMARY KEY,
      typename TEXT
    );
    """,
    """
    CREATE TABLE showXtype (
      show_id INTEGER REFERENCES show(show_id),
      type_id INTEGER REFERENCES type(type_id),
      PRIMARY KEY (show_id, type_id)
    );
    """
]

for table_sql in crear_tablas:
    cursor.execute(table_sql)

conn.commit()

<h3>Carga de datos</h3>

<p>No hay drama con cargar directamente estos datos</p>

In [ ]:
plataformas.head()

In [ ]:
plataformas.to_sql ('show', conn, index=False, if_exists='replace')
plataformas.to_sql ('platform', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXplatform', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXdirector', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXcast', conn, index=False, if_exists='replace')

plataformas.to_sql ('showXcountry', conn, index=False, if_exists='replace')
plataformas.to_sql ('rating', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXrating', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXlisted_in', conn, index=False, if_exists='replace')
plataformas.to_sql ('type', conn, index=False, if_exists='replace')
plataformas.to_sql ('showXtype', conn, index=False, if_exists='replace')

<p>Ahora restaria cargar los datos, para esto voy a tener que manipular primero el dataframe plataformas de forma conveniente para despues recorrer con un diccionario la carga de datos</p>

<p>Una vez manipulados los dataframes de forma conveniente para esta parte, con un diccionario los recorro y de cargo los datos</p>

In [ ]:
plataformas.to_sql ('director', conn, index=False, if_exists='replace')

In [ ]:
plataformas.to_sql ('cast', conn, index=False, if_exists='replace')

In [ ]:
plataformas.to_sql ('listed_in', conn, index=False, if_exists='replace')

In [ ]:
plataformas.to_sql ('country', conn, index=False, if_exists='replace')

<h3>#Ej de como podria normalizar estas tablas:</h3>

In [ ]:
df_director = plataformas["director"]
df_director.head()

In [ ]:
df_director = df_director.explode()
df_director.head()

In [ ]:
conn.commit()
conn.close()

<p>Al terminar, cierro la base de datos. Esta lista la parte funcional del programa</p>
<p>Lo unico que no me gusta de la base de datos es que si por ejemplo imprimo los directores, todavia quedaron listas adentro, no me va a dar el tiempo para arreglar esos fallos en la normalizacion</p>

In [ ]:
plataformas.head()

In [ ]:
#pd.read_sql('SELECT * FROM showXdirector', conn)

<h2>4. SQL</h2>

<p>Abro la base de datos para hacer las consultas</p>

In [ ]:
conn = sqlite3.connect(base_datos)
cursor = conn.cursor()

<p>Las consultas las podria hacer con pandas</p>

<p>o desde la interfaz grafica ya que para las pruebas fui mirando con DBeaver</p>

<img src='fotos/Dbeaver.png'>

<p>1. Considerando únicamente la plataforma de Netflix, ¿qué actor aparece
más veces? </p>

In [ ]:
query_drop_show_cast_view = "DROP VIEW IF EXISTS show_cast;"
conn.execute(query_drop_show_cast_view)

query_create_show_cast_view = f"""
CREATE VIEW show_cast AS
SELECT s.show_id, s.title, c.cast AS actor_name
FROM show AS s
JOIN showXcast AS sc ON s.show_id = sc.show_id
JOIN cast AS c ON sc.cast_id = c.cast_id;
"""

conn.execute(query_create_show_cast_view)


In [ ]:
query_drop_actor_appearances_view = "DROP VIEW IF EXISTS actor_appearances;"
conn.execute(query_drop_actor_appearances_view)

query_create_actor_appearances_view = f"""
CREATE VIEW actor_appearances AS
SELECT actor_name, COUNT(show_id) AS appearances
FROM show_cast
GROUP BY actor_name;
"""

conn.execute(query_create_actor_appearances_view)

In [ ]:

query_find_most_appeared_actor = f"""
SELECT actor_name, appearances
FROM actor_appearances
WHERE appearances = (SELECT MAX(appearances) FROM actor_appearances);
"""

result = pd.read_sql(query_find_most_appeared_actor, conn)


#  función para convertir el formato de la columna date_added a un tipo de dato compatible con sqlite
def convert_date(date):
  # Si la fecha es nula, devolvemos None
  if date is None:
    return None
  # Si la fecha tiene el formato Mes Dia, Año, la convertimos a Año-Mes-Dia
  elif len(date.split()) == 3:
    month, day, year = date.split()
    month = month[:-1] # Eliminamos la coma del mes
    # Creamos un diccionario con los nombres de los meses en inglés y sus números correspondientes
    months = {'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12'}
    # Devolvemos la fecha en el formato Año-Mes-Dia
    return f'{year}-{months[month]}-{day}'
  # Si la fecha tiene otro formato, la devolvemos tal cual
  else:
    return date

# Creamos una función para separar la lista de valores por cada celda y devolver una lista de tuplas
def split_list(values):
  # Si los valores son nulos, devolvemos una lista vacía
  if values is None:
    return []
  # Si los valores son una cadena, la separamos por comas y eliminamos los espacios en blanco
  elif isinstance(values, str):
    values = values.split(',')
    values = [value.strip() for value in values]
  # Si los valores son una lista, los dejamos tal cual
  elif isinstance(values, list):
    pass
  # Creamos una lista de tuplas con los valores y sus índices
  tuples = [(i+1, value) for i, value in enumerate(values)]
  # Devolvemos la lista de tuplas
  return tuples

# Obtenemos los datos de nuestro dataframe plataformas
data = plataformas.to_dict('records')

# Recorremos cada fila de los datos
for row in data:
  # Obtenemos los valores de cada columna
  show_id = row['show_id']
  type = row['type']
  title = row['title']
  director = row['director']
  cast = row['cast']
  country = row['country']
  date_added = row['date_added']
  release_year = row['release_year']
  rating = row['rating']
  duration = row['duration']
  listed_in = row['listed_in']
  description = row['description']
  platform = row['platform']

  # Convertimos el formato de la columna date_added
  date_added = convert_date(date_added)

  # Separamos la lista de directores, actores, países y géneros
  director = split_list(director)
  cast = split_list(cast)
  country = split_list(country)
  listed_in = split_list(listed_in)


<p>Finalmente, cierro la base de datos para liberar recursos</p>

In [ ]:
conn.close()